In [131]:
import torch

In [157]:
class Dense:
  def __init__(self, input, output, activation = lambda x: x):
    self.input = input
    self.output = output
    self.activation = activation
    self.weight = torch.rand(input, output, requires_grad=True)
    self.bias = torch.zeros(output, requires_grad=True)
    self.parameters = [self.weight, self.bias]
  
  def forward(self, x):
    return self.activation(x.matmul(self.weight) + self.bias)

In [158]:
linear = Dense(10, 2)
X = torch.ones((1, 10))
target = torch.tensor([[42. ,67.]])
loss_fn = torch.nn.MSELoss()
for i in range(2):
    pred = linear.forward(X)
    loss = loss_fn(pred, target)
    loss.backward()
    with torch.no_grad():
        for param in linear.parameters:
            param -= param.grad * 0.1
            param.grad.zero_()

with torch.no_grad():
    y = linear.forward(X)
print(y)

tensor([[41.6243, 66.3802]])


In [159]:
class MLP:
    def __init__(self, arch): #arch is a list tuples of size 3 corresponding to the layers
        self.arch = arch
        self.layers = []
        for i in range(len(arch)):
            self.layers.append(Dense(arch[i][0], arch[i][1], arch[i][2]))
        self.parameters = []
        for layer in self.layers:
            self.parameters.append(layer.parameters[0])
            self.parameters.append(layer.parameters[1])

    def forward(self, x):
        for layer in self.layers:
            x = layer.forward(x)
        return x 

In [160]:
mlp = MLP([(10, 20, torch.relu), (20, 10, torch.relu), (10, 2, lambda x: x)])
mlp.forward(torch.ones(1,10))
target = torch.tensor([42.,67.])
loss_fn = torch.nn.MSELoss()
for i in range(150):
    pred = mlp.forward(torch.ones(1,10))
    loss = loss_fn(pred, target)
    loss.backward()
    with torch.no_grad():
        for param in mlp.parameters:
            param -= param.grad * 0.1
            param.grad.zero_()
with torch.no_grad():
    y = mlp.forward(torch.ones(1,10))
print(y)

tensor([[42.0000, 67.0000]])


/usr/local/lib/python3.12/dist-packages/torch/nn/modules/loss.py:634: UserWarning: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([1, 2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
